# LDA Topic Modeling on Named Entities

In [1]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [2]:
df = pd.read_csv('Cleaned_Data.csv')

In [60]:
df_sub = df[['content','publication','sentiments']]

In [61]:
df_sub = df_sub.sample(frac= .1).reset_index()

In [67]:
print(len(df_sub))
print(len(df))

13510
135097


### Computing NLP annotations not possible for data of this size, so just extracting entities for a random sample

In [68]:
df_sub.entities = df_sub.content.progress_apply(lambda x: nlp(x).ents)

100%|██████████████████████████████████| 13510/13510 [1:51:29<00:00,  2.45it/s]


In [159]:
df_sub['entities'] = df_sub.entities

In [160]:
df_sub.head()

,index,content,publication,sentiments,entities
0,97145,The Japanese air bag manufacturer Takata has r...,NPR,-0.9791,"((Japanese), (Takata), ($, 1, billion), (the, ..."
1,72518,Can’t imagine pitying the bank account of an N...,New York Post,0.9792,"((NFL), (’s), (the, Super, Bowl), (a, few, yea..."
2,82968,"With summer nearing, my thoughts turn to the C...",New York Post,0.9849,"((summer), (the, Clam, Bar), (Montauk, Highway..."
3,110360,"Last week, President Donald J. Trump chose th...",Reuters,-0.9975,"((Last, week), (Donald, J., Trump), (U., S.), ..."
4,99964,The Marine Corps has released a recruiting ad ...,NPR,-0.7496,"((The, Marine, Corps), (first), (Marine), (Cor..."


In [161]:
df_sub['entities'] = df_sub['entities'].progress_apply(lambda x: list(x))

100%|█████████████████████████████████| 13510/13510 [00:00<00:00, 22343.38it/s]


In [162]:
df_sub['entities'] = df_sub['entities'].progress_apply(lambda x: [str(word) for word in x])

100%|███████████████████████████████████| 13510/13510 [00:42<00:00, 315.23it/s]


In [163]:
for x in range(len(df_sub['entities'])):
    good = []
    for word in df_sub['entities'][x]:
        word = str(word)
        word = word.lower()
        if word not in ['one','two','three','four','five','six','seven','eight','nine','ten']:
            if word.isdigit() == False:
                word = word.strip()
                if len(str(word)) > 3:
                    good += [word]

    if x%1000 == 0:
        print(x)

    df_sub['entities'][x] = good

0


C:\Users\Henry\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000


In [170]:
def hasNumbers(inputString):
...     return any(char.isdigit() for char in inputString)

In [173]:
for x in range(len(df_sub['entities'])):
    good = []
    for word in df_sub['entities'][x]:
        if hasNumbers(word) == False:
            good += [word]
    if x%1000 == 0:
        print(x)
    df_sub['entities'][x] = good

0


C:\Users\Henry\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000


In [174]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [175]:
def dummy(doc):
    return doc

In [176]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [179]:
def pyLDA(pub, num_topics=20):
    tf_vectorizer = CountVectorizer(max_df = 0.8, min_df = 2, tokenizer=dummy,preprocessor=dummy)
    dtm_tf = tf_vectorizer.fit_transform(pub)
    lda_tf = LatentDirichletAllocation(num_topics, batch_size = 200, evaluate_every=-1, learning_method='online', learning_offset = 10.0)
    lda_tf.fit(dtm_tf)
    return pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='mmds')

## Recommend lowering relevance metric to 0.2 - 0.5

In [180]:
pyLDA(df_sub['entities'])

C:\Users\Henry\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.329556 -0.267616       1        1  18.371876
6      0.352320 -0.103561       2        1  15.142629
19     0.181192 -0.234647       3        1  12.770728
1      0.025850 -0.329130       4        1   7.478184
5      0.172239 -0.419573       5        1   5.598845
4     -0.293153 -0.305871       6        1   5.165276
17    -0.060838 -0.083270       7        1   5.006862
8      0.138891  0.021946       8        1   4.630270
0      0.007220  0.176130       9        1   3.736133
14    -0.153140 -0.370772      10        1   2.590364
18     0.418254  0.068800      11        1   2.520480
10    -0.080630  0.355717      12        1   2.428497
15     0.199739  0.289186      13        1   2.367991
7     -0.274959  0.354198      14        1   2.198806
9      0.087909  0.449312      15        1   2.189105
12    -0.226057  0.165909      16        1   1.989398
3     -0.315776 -0.045545      17        1   1.638317
16    -0.404285  0.139324      18        1   1.595678
11     0.341611  0.262390      19        1   1.331830
13    -0.445943 -0.122927      20        1   1.248731, topic_info=      Category          Freq               Term         Total  loglift  \
term                                                                     
27775  Default  28146.000000              trump  28146.000000  30.0000   
4987   Default   8767.000000            clinton   8767.000000  29.0000   
27847  Default   5763.000000            trump’s   5763.000000  28.0000   
19832  Default   4105.000000        republicans   4105.000000  27.0000   
1350   Default   3269.000000          americans   3269.000000  26.0000   
6295   Default   3558.000000          democrats   3558.000000  25.0000   
19824  Default   5731.000000         republican   5731.000000  24.0000   
21288  Default   2512.000000             senate   2512.000000  23.0000   
10427  Default   2013.000000              house   2013.000000  22.0000   
28099  Default   4949.000000              u. s.   4949.000000  21.0000   
8178   Default   9638.000000              first   9638.000000  20.0000   
10196  Default   2994.000000    hillary clinton   2994.000000  19.0000   
7850   Default   1601.000000           facebook   1601.000000  18.0000   
5223   Default   2672.000000           congress   2672.000000  17.0000   
4639   Default   2717.000000              china   2717.000000  16.0000   
20600  Default   3478.000000             russia   3478.000000  15.0000   
17226  Default   2630.000000           new york   2630.000000  14.0000   
10972  Default   1110.000000             israel   1110.000000  13.0000   
20897  Default   1648.000000            sanders   1648.000000  12.0000   
22892  Default   2160.000000              syria   2160.000000  11.0000   
6669   Default   5319.000000       donald trump   5319.000000  10.0000   
6288   Default   2908.000000         democratic   2908.000000   9.0000   
7722   Default   1516.000000             europe   1516.000000   8.0000   
15807  Default   1325.000000             mexico   1325.000000   7.0000   
5518   Default   1795.000000               cruz   1795.000000   6.0000   
26600  Default   4656.000000  the united states   4656.000000   5.0000   
3597   Default   1188.000000            britain   1188.000000   4.0000   
1308   Default   5571.000000           american   5571.000000   3.0000   
17616  Default    960.000000        north korea    960.000000   2.0000   
23238  Default   1428.000000              texas   1428.000000   1.0000   
...        ...           ...                ...           ...      ...   
11062  Topic20     49.258749               jack     50.151043   4.3651   
19555  Topic20     48.186541            ramirez     49.078755   4.3647   
18714  Topic20     40.454594          perlstein     41.346920   4.3612   
3621   Topic20     38.686722         broaddrick     39.578936   4.3602   
29437  Topic20     38.447558   